In [ ]:
CREATE ROLE ADMIN;
CREATE ROLE DEVELOPER;
CREATE ROLE PII;

GRANT ROLE ACCOUNTADMIN TO ROLE ADMIN;
GRANT ROLE ACCOUNTADMIN TO ROLE PII;
GRANT ROLE ADMIN TO ROLE DEVELOPER;

GRANT ROLE ADMIN TO USER AXSHK;
GRANT ROLE PII TO USER AXSHK;
GRANT ROLE DEVELOPER TO USER AXSHK;

CREATE TABLE employees_csv (
    EMPLOYEE_ID INT PRIMARY KEY,
    FIRST_NAME VARCHAR(50),
    LAST_NAME VARCHAR(50),
    EMAIL VARCHAR(100),
    PHONE_NUMBER VARCHAR(20),
    HIRE_DATE DATE,
    JOB_ID VARCHAR(10),
    SALARY DECIMAL(10, 2),
    COMMISSION_PCT DECIMAL(5, 2),
    MANAGER_ID INT,
    DEPARTMENT_ID INT,
    ADDRESS VARCHAR(255)
);

CREATE OR REPLACE STAGE employees_data_int_stage;
ls @employees_data_int_stage;

CREATE OR REPLACE FILE FORMAT infer_csv_format
TYPE = CSV
COMPRESSION = GZIP
FIELD_DELIMITER = ','
PARSE_HEADER = TRUE
DATE_FORMAT = 'YYYY-MM-DD'
FIELD_OPTIONALLY_ENCLOSED_BY = '"'

SELECT * FROM TABLE(INFER_SCHEMA(
 LOCATION=>'@employees_data_int_stage/employees2.csv.gz'
 , FILE_FORMAT=>'infer_csv_format'));

 CREATE OR REPLACE TABLE employees_csv USING TEMPLATE (
SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) 
 WITHIN GROUP (ORDER BY ORDER_ID)
 FROM TABLE (INFER_SCHEMA(
 LOCATION=>'@employees_data_int_stage/employees2.csv.gz',
 FILE_FORMAT=>'infer_csv_format')));

 SELECT GET_DDL('table', 'employees_csv');

ALTER TABLE employees_csv ADD COLUMN elt_by VARCHAR(100);
ALTER TABLE employees_csv ADD COLUMN elt_ts TIMESTAMP_LTZ;
ALTER TABLE employees_csv ADD COLUMN file_name VARCHAR(100);

CREATE OR REPLACE FILE FORMAT read_csv_format
TYPE = CSV
FIELD_DELIMITER = ','
SKIP_HEADER = 1
FIELD_OPTIONALLY_ENCLOSED_BY = '"'
EMPTY_FIELD_AS_NULL = TRUE

COPY INTO employees_csv (
    EMPLOYEE_ID,
	FIRST_NAME,
	LAST_NAME,
	EMAIL,
	PHONE_NUMBER,
	HIRE_DATE,
	JOB_ID,
	SALARY,
	COMMISSION_PCT,
	MANAGER_ID,
	DEPARTMENT_ID,
	ADDRESS,
    ELT_BY,
    ELT_TS,
    FILE_NAME
)
FROM (
    SELECT $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12, 'my_app_name' AS ELT_BY, CURRENT_TIMESTAMP AS ELT_TS, METADATA$FILENAME AS FILE_NAME FROM '@employees_data_int_stage/employees2.csv.gz' (file_format => 'read_csv_format')
)
ON_ERROR = SKIP_FILE;

select * from employees_csv;

CREATE OR REPLACE TABLE employees_variant(
employee_data VARIANT
);

INSERT INTO employees_variant(
SELECT TO_VARIANT(OBJECT_CONSTRUCT(*))
FROM employees_csv
);

SELECT * FROM employees_variant;

CREATE OR REPLACE STORAGE INTEGRATION s3_integration
TYPE = EXTERNAL_STAGE
STORAGE_PROVIDER = 'S3'
STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::545009850146:role/Aashu_AWS'
ENABLED = TRUE
STORAGE_ALLOWED_LOCATIONS = ("s3://snowflake-sigmoid-bucket/");

DESCRIBE INTEGRATION s3_integration;

CREATE OR REPLACE STAGE employees_data_ext_stage
URL = 's3://newbucketdottxt/employees.csv'
STORAGE_INTEGRATION = s3_integration;

list @employees_data_ext_stage;

CREATE OR REPLACE TABLE employees_external(
	EMPLOYEE_ID NUMBER(3,0),
	FIRST_NAME VARCHAR(16777216),
	LAST_NAME VARCHAR(16777216),
	EMAIL VARCHAR(16777216),
	PHONE_NUMBER VARCHAR(16777216),
	HIRE_DATE DATE,
	JOB_ID VARCHAR(16777216),
	SALARY NUMBER(5,0),
	COMMISSION_PCT NUMBER(3,2),
	MANAGER_ID NUMBER(3,0),
	DEPARTMENT_ID NUMBER(3,0),
	ADDRESS VARCHAR(16777216),
    elt_by VARCHAR(100),
    elt_ts TIMESTAMP_LTZ,
    file_name varchar(100)
);

COPY INTO employees_external
FROM (
    SELECT $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12, 'my_app_name' AS ELT_BY, CURRENT_TIMESTAMP AS ELT_TS, METADATA$FILENAME AS FILE_NAME FROM '@employees_data_int_stage/employees2.csv.gz'
)
FILE_FORMAT = (skip_header = 1, field_optionally_enclosed_by = '"')
ON_ERROR = SKIP_FILE;

SELECT * FROM employees_external;

CREATE OR REPLACE FILE FORMAT infer_parquet_format
TYPE = PARQUET
COMPRESSION = AUTO
USE_LOGICAL_TYPE = TRUE
TRIM_SPACE = TRUE
REPLACE_INVALID_CHARACTERS = TRUE
NULL_IF = ( '\N', 'NULL', 'NUL', '' );

SELECT * FROM TABLE(INFER_SCHEMA(
 LOCATION=>'@employees_data_ext_stage/titanic.parquet'
 , FILE_FORMAT=>'infer_parquet_format'
 , MAX_RECORDS_PER_FILE => 10));

 SELECT *, 'my_app_name' AS ELT_BY, CURRENT_TIMESTAMP AS ELT_TS, METADATA$FILENAME AS FILE_NAME FROM '@employees_data_ext_stage/titanic.parquet' (FILE_FORMAT => infer_parquet_format);

 CREATE OR REPLACE MASKING POLICY pii_mask AS (val string) RETURNS string ->
CASE
    WHEN current_role() IN ('DEVELOPER') THEN '****'
    ELSE val
END;

ALTER TABLE IF EXISTS employees_csv MODIFY COLUMN email SET MASKING POLICY pii_mask;
ALTER TABLE IF EXISTS employees_csv MODIFY COLUMN address SET MASKING POLICY pii_mask;
ALTER TABLE IF EXISTS employees_csv MODIFY COLUMN phone_number SET MASKING POLICY pii_mask

use role DEVELOPER;

SELECT * FROM employees_csv;